In [2]:
import yfinance as yf
import pandas as pd

# Fetch historical data for an interest rate proxy, e.g., 10-year U.S. Treasury yield
ticker = '^TNX'  # Symbol for the 10-year U.S. Treasury yield (in percentage)
data = yf.download(ticker, start='2010-01-01', end='2024-01-01')

# Use the 'Adj Close' as the interest rate (percentage)
interest_rates = data['Adj Close']

# Normalize the interest rates for GAN
interest_rates_normalized = (interest_rates - interest_rates.mean()) / interest_rates.std()
interest_rates_normalized = interest_rates_normalized.values.reshape(-1, 1)


[*********************100%***********************]  1 of 1 completed


In [4]:
interest_rates_normalized

array([[1.75146007],
       [1.64906944],
       [1.71217064],
       ...,
       [1.68954938],
       [1.76217517],
       [1.78122464]])

In [10]:
latent_dim = 100  # Dimensionality of the latent space (input to the generator)


In [14]:
!pip install tensorflow


  Using cached tensorflow-2.18.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
Using cached tensorflow-2.18.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (615.5 MB)


In [15]:
import yfinance as yf
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Define latent dimension
latent_dim = 100  # Latent space dimensionality

# Fetch historical data for the 10-year U.S. Treasury yield (interest rate proxy)
ticker = '^TNX'  # Symbol for the 10-year U.S. Treasury yield (in percentage)
data = yf.download(ticker, start='2010-01-01', end='2024-01-01')

# Use the 'Adj Close' as the interest rate (percentage)
interest_rates = data['Adj Close']

# Normalize the interest rates for GAN
interest_rates_normalized = (interest_rates - interest_rates.mean()) / interest_rates.std()
interest_rates_normalized = interest_rates_normalized.values.reshape(-1, 1)

# Build the Generator
def build_generator(latent_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='tanh'))  # Output is one interest rate value
    return model

# Build the Discriminator
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(1,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Output probability of real or fake
    return model

# Build the GAN Model (Combining Generator and Discriminator)
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Compile models
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

generator = build_generator(latent_dim)
generator.compile(loss='binary_crossentropy', optimizer='adam')

gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Hyperparameters
batch_size = 32
epochs = 10000
half_batch = batch_size // 2

# Training loop
for epoch in range(epochs):
    # Train discriminator with real and fake data
    idx = np.random.randint(0, interest_rates_normalized.shape[0], half_batch)
    real_rates = interest_rates_normalized[idx]

    noise = np.random.normal(0, 1, (half_batch, latent_dim))
    fake_rates = generator.predict(noise)

    # Labels for real and fake data
    real_labels = np.ones((half_batch, 1))
    fake_labels = np.zeros((half_batch, 1))

    # Train the discriminator (real vs fake)
    d_loss_real = discriminator.train_on_batch(real_rates, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_rates, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator via the GAN model (fooling the discriminator)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))  # We want to fool the discriminator into thinking it's real
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print the progress
    if epoch % 1000 == 0:
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {d_loss[1]*100}%] [G loss: {g_loss}]")

# Generate synthetic interest rates after training
noise = np.random.normal(0, 1, (1000, latent_dim))  # Generate synthetic samples
synthetic_interest_rates = generator.predict(noise)

# Rescale back to original interest rate scale
synthetic_interest_rates_rescaled = synthetic_interest_rates * interest_rates.std() + interest_rates.mean()

# Plot real vs synthetic interest rates
plt.figure(figsize=(10, 6))
plt.plot(interest_rates[:1000], label='Real Interest Rates')
plt.plot(synthetic_interest_rates_rescaled, label='Generated Interest Rates', linestyle='dashed')
plt.legend()
plt.show()


[*********************100%***********************]  1 of 1 completed
/home/j/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-01-05 00:08:45.400704: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step


AttributeError: 'NoneType' object has no attribute 'update_state'

In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define latent dimension for the generator input
latent_dim = 100

# Fetch historical data for the 10-year U.S. Treasury yield (interest rate proxy)
ticker = '^TNX'  # Symbol for the 10-year U.S. Treasury yield (in percentage)
data = yf.download(ticker, start='2010-01-01', end='2024-01-01')

# Use the 'Adj Close' as the interest rate (percentage)
interest_rates = data['Adj Close']

# Normalize the interest rates for GAN
interest_rates_normalized = (interest_rates - interest_rates.mean()) / interest_rates.std()
interest_rates_normalized = interest_rates_normalized.values.reshape(-1, 1)

# Build the Generator model
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='tanh'))  # Output is one interest rate value
    return model

# Build the Discriminator model
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(1,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Output probability of real or fake
    return model

# Build the GAN (combining generator and discriminator)
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze discriminator when training the generator
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Compile the models
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

generator = build_generator(latent_dim)

# The GAN model combines the generator and the discriminator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Hyperparameters
batch_size = 32
epochs = 10000
half_batch = batch_size // 2

# Training loop
for epoch in range(epochs):
    # Train discriminator with real and fake data
    idx = np.random.randint(0, interest_rates_normalized.shape[0], half_batch)
    real_rates = interest_rates_normalized[idx]

    noise = np.random.normal(0, 1, (half_batch, latent_dim))
    fake_rates = generator.predict(noise)

    # Labels for real and fake data
    real_labels = np.ones((half_batch, 1))
    fake_labels = np.zeros((half_batch, 1))

    # Train the discriminator (real vs fake)
    d_loss_real = discriminator.train_on_batch(real_rates, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_rates, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator via the GAN model (fooling the discriminator)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))  # We want to fool the discriminator into thinking it's real
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print the progress
    if epoch % 1000 == 0:
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {d_loss[1]*100}%] [G loss: {g_loss}]")

# Generate synthetic interest rates after training
noise = np.random.normal(0, 1, (1000, latent_dim))  # Generate synthetic samples
synthetic_interest_rates = generator.predict(noise)

# Rescale back to original interest rate scale
synthetic_interest_rates_rescaled = synthetic_interest_rates * interest_rates.std() + interest_rates.mean()

# Plot real vs synthetic interest rates
plt.figure(figsize=(10, 6))
plt.plot(interest_rates[:1000], label='Real Interest Rates')
plt.plot(synthetic_interest_rates_rescaled, label='Generated Interest Rates', linestyle='dashed')
plt.legend()
plt.show()


[*********************100%***********************]  1 of 1 completed
/home/j/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


AttributeError: 'NoneType' object has no attribute 'update_state'

In [27]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define latent dimension for the generator input
latent_dim = 100

# Fetch historical data for the 10-year U.S. Treasury yield (interest rate proxy)
ticker = '^TNX'  # Symbol for the 10-year U.S. Treasury yield (in percentage)
data = yf.download(ticker, start='2010-01-01', end='2024-01-01')

# Use the 'Adj Close' as the interest rate (percentage)
interest_rates = data['Adj Close']

# Normalize the interest rates for GAN
interest_rates_normalized = (interest_rates - interest_rates.mean()) / interest_rates.std()
interest_rates_normalized = interest_rates_normalized.values.reshape(-1, 1)

# Build the Generator model
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='tanh'))  # Output is one interest rate value
    return model

# Build the Discriminator model
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(1,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Output probability of real or fake
    return model

# Build the GAN (combining generator and discriminator)
def build_gan(generator, discriminator):
    # Freeze discriminator when training the generator
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Compile the models
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

generator = build_generator(latent_dim)

# The GAN model combines the generator and the discriminator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Hyperparameters
batch_size = 32
epochs = 10000
half_batch = batch_size // 2


[*********************100%***********************]  1 of 1 completed
/home/j/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [35]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define latent dimension for the generator input
latent_dim = 100

# Fetch historical data for the 10-year U.S. Treasury yield (interest rate proxy)
ticker = '^TNX'  # Symbol for the 10-year U.S. Treasury yield (in percentage)
data = yf.download(ticker, start='2010-01-01', end='2024-01-01')

# Use the 'Adj Close' as the interest rate (percentage)
interest_rates = data['Adj Close']

# Normalize the interest rates for GAN
interest_rates_normalized = (interest_rates - interest_rates.mean()) / interest_rates.std()
interest_rates_normalized = interest_rates_normalized.values.reshape(-1, 1)

# Build the Generator model
def build_generator(latent_dim):
  model = models.Sequential()
  model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1, activation='tanh'))  # Output is one interest rate value
  return model

# Build the Discriminator model
def build_discriminator():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu', input_shape=(1,)))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))  # Output probability of real or fake
  return model

# Build the GAN (combining generator and discriminator)
def build_gan(generator, discriminator):
  # Freeze discriminator when training the generator
  discriminator.trainable = False
  model = models.Sequential()
  model.add(generator)
  model.add(discriminator)
  return model

# Compile the models
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

generator = build_generator(latent_dim)

# The GAN model combines the generator and the discriminator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Hyperparameters
batch_size = 32
epochs = 10000
half_batch = batch_size // 2



[*********************100%***********************]  1 of 1 completed
/home/j/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
# Training loop
for epoch in range(epochs):
  # Train discriminator with real and fake data
  idx = np.random.randint(0, interest_rates_normalized.shape[0], half_batch)
  real_rates = interest_rates_normalized[idx]

  noise = np.random.normal(0, 1, (half_batch, latent_dim))
  fake_rates = generator.predict(noise)

  # Labels for real and fake data (ensure the shape is (half_batch, 1))
  real_labels = np.ones((half_batch, 1))
  fake_labels = np.zeros((half_batch, 1))

  # Train the discriminator (real vs fake)
  d_loss_real = discriminator.train_on_batch(real_rates, real_labels)
  d_loss_fake = discriminator.train_on_batch(fake_rates, fake_labels)
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  # Train the generator via the GAN model (fooling the discriminator)
  noise = np.random.normal(0, 1, (batch_size, latent_dim))
  valid_labels = np.ones((batch_size, 1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step


AttributeError: 'NoneType' object has no attribute 'update_state'

In [31]:
# Compile the models (ensure this happens before training)
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
generator = build_generator(latent_dim)

# The GAN model combines the generator and the discriminator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Hyperparameters
batch_size = 32
epochs = 10000
half_batch = batch_size // 2

# Training loop
for epoch in range(epochs):
     # Train discriminator with real and fake data
  idx = np.random.randint(0, interest_rates_normalized.shape[0], half_batch)
  real_rates = interest_rates_normalized[idx]

  noise = np.random.normal(0, 1, (half_batch, latent_dim))
  fake_rates = generator.predict(noise)

  # Labels for real and fake data (ensure the shape is (half_batch, 1))
  real_labels = np.ones((half_batch, 1))
  fake_labels = np.zeros((half_batch, 1))

  # Train the discriminator (real vs fake)
  d_loss_real = discriminator.train_on_batch(real_rates, real_labels)
  d_loss_fake = discriminator.train_on_batch(fake_rates, fake_labels)
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  # Train the generator via the GAN model (fooling the discriminator)
  noise = np.random.normal(0, 1, (batch_size, latent_dim))
  valid_labels = np.ones((batch_size, 1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step


AttributeError: 'NoneType' object has no attribute 'update_state'

In [39]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# ... (data loading and preprocessing as before)

# Build the Generator model (no changes)
def build_generator(latent_dim):
    # ... (same as before)
    return model

# Build the Discriminator model (no changes)
def build_discriminator():
    # ... (same as before)
    return model

# Build the GAN (combining generator and discriminator)
def build_gan(generator, discriminator):
    # Freeze discriminator when training the generator *BEFORE COMPILING THE GAN*
    discriminator.trainable = False  # Crucial: Freeze here!
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Create the discriminator and generator
discriminator = build_discriminator()
generator = build_generator(latent_dim)

# Create and compile the GAN *AFTER* freezing the discriminator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Compile the discriminator separately
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# ... (rest of the training loop and plotting as before)

NameError: name 'model' is not defined

In [41]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# ... (data loading and preprocessing as before)

# Build the Generator model
def build_generator(latent_dim):
    model = models.Sequential() # The missing line!
    model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='tanh'))
    return model

# Build the Discriminator model
def build_discriminator():
    model = models.Sequential() # The missing line!
    model.add(layers.Dense(64, activation='relu', input_shape=(1,)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Build the GAN (combining generator and discriminator)
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Create the discriminator and generator
discriminator = build_discriminator()
generator = build_generator(latent_dim)

# Create and compile the GAN *AFTER* freezing the discriminator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Compile the discriminator separately
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

generator = build_generator(latent_dim)

# The GAN model combines the generator and the discriminator
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Hyperparameters
batch_size = 32
epochs = 10000
half_batch = batch_size // 2

for epoch in range(epochs):
  # Train discriminator with real and fake data
  idx = np.random.randint(0, interest_rates_normalized.shape[0], half_batch)
  real_rates = interest_rates_normalized[idx]

  noise = np.random.normal(0, 1, (half_batch, latent_dim))
  fake_rates = generator.predict(noise)

  # Labels for real and fake data (ensure the shape is (half_batch, 1))
  real_labels = np.ones((half_batch, 1))
  fake_labels = np.zeros((half_batch, 1))

  # Train the discriminator (real vs fake)
  d_loss_real = discriminator.train_on_batch(real_rates, real_labels)
  d_loss_fake = discriminator.train_on_batch(fake_rates, fake_labels)
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  # Train the generator via the GAN model (fooling the discriminator)
  noise = np.random.normal(0, 1, (batch_size, latent_dim))
  valid_labels = np.ones((batch_size, 1))

In [45]:
# Training loop


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


AttributeError: 'NoneType' object has no attribute 'update_state'